In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
import xgboost as xgb
from wheather_data_prediction import calculate_POA, predict_linear_reg, lstm_atten_train, cnn_lstm_atten_train

2024-07-30 09:49:59.771630: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-30 09:49:59.983718: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-30 09:50:02.937601: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
#Reading NASA dataset from exact NEOEN site 
# Read the CSV data into a Pandas DataFrame
from io import StringIO
df_met_nasa = pd.read_csv('/mnt/data-r6/ahilan/dataset/neoen/process/NASA-weather-data-sattelite_20200101_20240131_034d76S_145d93E_LST.csv',skiprows=16)
df_met_nasa = df_met_nasa.rename(columns={'MO': 'MONTH','DY': 'DAY', 'HR': 'HOUR' , 'ALLSKY_SFC_SW_DWN' : 'GHI',  'ALLSKY_SFC_SW_DNI': 'DNI', 'ALLSKY_SFC_SW_DIFF' : 'DHI', 'T2M': 'Temp', 'WS10M': 'Wind', 'QV2M': 'HUMDY', 'ALLSKY_SRF_ALB' : 'ALBEDO', 'PRECTOTCORR': 'PRECIP'})
df_met_nasa['datetime'] = pd.to_datetime(df_met_nasa[['YEAR', 'MONTH', 'DAY', 'HOUR']])
df_met_nasa.drop(columns=['YEAR', 'MONTH', 'DAY', 'HOUR'], inplace=True)
df_met_nasa.set_index('datetime', inplace=True)

lat, lon = -34.76, 145.93
ghi = df_met_nasa['GHI']
dni = df_met_nasa['DNI']
dhi = df_met_nasa['DHI']
albedo=df_met_nasa['ALBEDO']

azimuth_angle = 0  # azimuth angle (north-facing)

start_date = df_met_nasa.index.min()
end_date = df_met_nasa.index.max()

df_met_nasa['POA'] = calculate_POA(ghi,dni,dhi,albedo,lat,lon,azimuth_angle,start_date=start_date,end_date=end_date)

start_date = '2022-03-17'
end_date = '2022-03-20'


df_met_nasa['POA'][start_date:end_date].resample('H').mean().plot(figsize=(5, 4),label='POA')
df_met_nasa['GHI'][start_date:end_date].resample('H').mean().plot(figsize=(5, 4), label='GHI')
plt.legend()
plt.ylabel('Irradiance')
plt.xlabel('Date')
plt.grid(True)
plt.show()


df_met_nasa_temp = df_met_nasa[['Temp', 'POA', 'Wind','GHI','HUMDY']]
df_met_nasa_temp = df_met_nasa_temp.rename(columns={'Temp': 'Temp_nasa','POA': 'POA_nasa', 'Wind': 'Wind_nasa', 'GHI' : 'GHI_nasa','HUMDY': 'HUMDY_nasa'})

df_met_measured = pd.read_pickle('/mnt/data-r6/ahilan/dataset/neoen/process/met_5min.pkl')
df_met_measured = df_met_measured.rename(columns={'Timestamp': 'time'})
df_met_measured.set_index('time', inplace=True)

df_met_measured = df_met_measured.shift(periods=-90, freq='T')

df_met_measured = df_met_measured[['Temp','POA','Wind','Tmod']].resample('H').mean()
df_met_measured = df_met_measured.rename(columns={'Temp': 'Temp_mes','POA': 'POA_mes', 'Wind': 'Wind_mes', 'Tmod': 'Tmod_mes'})

data = pd.merge(df_met_measured, df_met_nasa_temp, left_index=True, right_index=True)


data = data[data['Temp_mes'] < 40]
data = data[data['Temp_mes'] > 0]


In [7]:
#start_date = data.index.min()
#end_date = data.index.max()

start_date = '2023-05-17'
end_date = '2023-05-20'

feat_nasa = 'POA_nasa'
feat_mes ='POA_mes'
label_nasa = 'Sat NASA POA '
label_mes = 'Measured POA'
ylabel = 'POA'

data[feat_nasa][start_date:end_date].resample('H').mean().plot(figsize=(5, 4),label=label_nasa)
data[feat_mes][start_date:end_date].resample('H').mean().plot(figsize=(5, 4),label=label_mes)
plt.legend()
plt.ylabel(ylabel)
plt.xlabel('Date')
plt.grid(True)
plt.show()

In [30]:
# Estimating the Sandia Module Temperature Model coefficnet U0 and U1
from sklearn.linear_model import LinearRegression

data_lr = data

# Ensure your dataset contains columns: 'T_m', 'T_a', 'v', 'G_POA'
T_m = data_lr['Tmod_mes']
T_a = data_lr['Temp_mes']
v = data_lr['Wind_mes']
G_POA = data_lr['POA_mes']
eta = 0.1737  # Efficiency

data_lr['Y'] = data_lr['POA_mes'] * (1 - eta)
data_lr['X1'] = data_lr['Tmod_mes'] - data_lr['Temp_mes']
data_lr['X2'] = data_lr['X1'] * data_lr['Wind_mes']

# Filter out infinite or NaN values
data_lr.replace([np.inf, -np.inf], np.nan, inplace=True)
data_lr.dropna(inplace=True)

# Define features and target
X = data_lr[['X1', 'X2']]
Y = data_lr['Y']


# Create a linear regression model
model = LinearRegression()

# Fit the model
model.fit(X, Y)

# Retrieve the coefficients
U0 = model.coef_[0]
#U1 = model.coef_[1]

print(f'U0: {U0}, U1: {U1}')

U0: 23.747483103376968, U1: 0.03351606


In [31]:
# Estimating module temperature using Sandia Module Temperature Model and ambient temperature, wind speed and POA
def estimate_module_temperature(T_amb, E, eta, U0, U1, v):
    return T_amb + (E * (1 - eta)) / (U0 + U1 * v)

# Example values
T_amb = df_met_nasa_temp['Temp_nasa']  # °C
E = df_met_nasa_temp['POA_nasa']    # W/m²
eta = 0.1737  # Efficiency
#U0 = 25     # W/m²K
U0 = 23.747483103376968
#U1 = 6.84   # W/m²K per m/s
U1 = 0.03351606
v = df_met_nasa_temp['Wind_nasa']  # m/s


df_met_nasa_temp['Tmod_nasa'] = estimate_module_temperature(T_amb, E, eta, U0, U1, v)

df_met_nasa = df_met_nasa_temp
print(df_met_nasa.columns)

#Saving the nasa date on pkl file
df_met_nasa.to_pickle('/home/ahilan/gen-research/neoen-site/data/neoen_weather_nasa_sat_data.pkl')

#start_date = data.index.min()
#end_date = data.index.max()

start_date = '2023-05-17'
end_date = '2023-05-20'

feat_nasa = 'Tmod_nasa'
feat_mes ='Tmod_mes'
label_nasa = 'Sat NASA Tmod '
label_mes = 'Measured Tmod'
ylabel = 'Tmod'

df_met_nasa_temp[feat_nasa][start_date:end_date].resample('H').mean().plot(figsize=(5, 4),label=label_nasa)
data[feat_mes][start_date:end_date].resample('H').mean().plot(figsize=(5, 4),label=label_mes)
plt.legend()
plt.ylabel(ylabel)
plt.xlabel('Date')
plt.grid(True)
plt.show()

Index(['Temp_nasa', 'POA_nasa', 'Wind_nasa', 'GHI_nasa', 'HUMDY_nasa',
       'Tmod_nasa'],
      dtype='object')


In [4]:
# Simple Linear regression modelling to predict actual temperature from satellite data temperature
data = pd.merge(df_met_measured, df_met_nasa_temp, left_index=True, right_index=True)
data = data[['Temp_mes','Wind_mes','POA_mes', 'POA_mes', 'Tmod_mes','Wind_nasa', 'Temp_nasa', 'POA_nasa', 'HUMDY_nasa','Tmod_nasa']]

#data = data[data['Wind_mes']<12]
#data = data[data['Wind_nasa']<12]

# Filtering constant values
# Define the window size
window_size = 3

# Check if the values in the rolling window are the same
data['constant'] = data['POA_mes'].rolling(window=window_size).apply(lambda x: np.all(x == x[0]), raw=True)

# Fill NaN values with 0 (occurs for the first 'window_size - 1' elements)
data['constant'].fillna(0, inplace=True)
data['constant'] = data['constant'].astype(bool)

# Filter out constant value rows
data_filtered = data[~data['constant']].drop(columns=['constant'])


data = data_filtered

feat_target = 'POA_mes'
sat_data = 'POA_nasa'
feat_pred = 'POA_pred'
y_label = 'POA'

# Split the data into features (X) and target (y)
X = data[['Wind_nasa', 'Temp_nasa', 'POA_nasa', 'HUMDY_nasa']].values
y = data[feat_target].values

y_pred = predict_linear_reg(X, y, feat_target, sat_data, feat_pred, y_label)

data[feat_pred] = y_pred


#start_date = data.index.min()
#end_date = data.index.max()

start_date = '2023-05-17'
end_date = '2023-05-20'

data[feat_target][start_date:end_date].resample('H').mean().plot(figsize=(5, 4),label=feat_target)
data[sat_data][start_date:end_date].resample('H').mean().plot(figsize=(5, 4),label=sat_data)
data[feat_pred][start_date:end_date].resample('H').mean().plot(figsize=(5, 4),label=feat_pred)
plt.legend()
plt.ylabel(y_label)
plt.xlabel('Date')
plt.grid(True)
plt.show()


Best parameters: {'regressor__copy_X': True, 'regressor__fit_intercept': True, 'regressor__n_jobs': None}
Best score: -20076.6564892879
Train MSE: 20046.093003103146
Test MSE: 20542.21884508547
Train R2: 0.859625419489919
Test R2: 0.8537038689548516


In [ ]:
# LSTM attention modelling to predict actual temperature from satellite data temperature
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

data = pd.merge(df_met_measured, df_met_nasa_temp, left_index=True, right_index=True)
data = data[['Temp_mes','Wind_mes','POA_mes', 'POA_mes', 'Tmod_mes','Wind_nasa', 'Temp_nasa', 'POA_nasa', 'HUMDY_nasa','Tmod_nasa']]

#data = data[data['Wind_mes']<12]
#data = data[data['Wind_nasa']<12]

# Filtering constant values
# Define the window size
window_size = 3

# Check if the values in the rolling window are the same
data['constant'] = data['Wind_mes'].rolling(window=window_size).apply(lambda x: np.all(x == x[0]), raw=True)

# Fill NaN values with 0 (occurs for the first 'window_size - 1' elements)
data['constant'].fillna(0, inplace=True)
data['constant'] = data['constant'].astype(bool)

# Filter out constant value rows
data_filtered = data[~data['constant']].drop(columns=['constant'])


data = data_filtered

start_date = data.index.min()
end_date = data.index.max()

feat_target = 'Wind_mes'
sat_data = 'Wind_nasa'
feat_pred = 'Wind_pred'
y_label = 'Wind'

# Split the data into features (X) and target (y)
X = data[['Wind_nasa', 'Temp_nasa', 'POA_nasa', 'HUMDY_nasa']].values
y = data[feat_target].values


# Normalize the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Prepare the data for LSTM (convert to sequences)
def create_sequences(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

seq_len = 24  # You can adjust this based on your data
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_len)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

model = lstm_atten_train(X_train, X_test, y_train, y_test, feat_target, sat_data, feat_pred, y_label,epochs=200)

# Evaluate the model
mse, mae = model.evaluate(X_test, y_test)
print(f"Test MSE: {mse}")
print(f"Test MAE: {mae}")

# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)


# Calculate metrics
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Train MSE: {train_mse}')
print(f'Test MSE: {test_mse}')
print(f'Train R2: {train_r2}')
print(f'Test R2: {test_r2}')

y_pred = scaler_y.inverse_transform(model.predict(X_seq))


plt.plot(y[24:84], color='blue', label=feat_target)
plt.plot(X[24:84,1], color='red', label=sat_data)
plt.plot(y_pred[0:60], color='green', label=feat_pred)
plt.title('Training Data')
plt.xlabel('Time')
plt.ylabel(y_label)
plt.legend()
plt.show()

# Plot the results
plt.figure(figsize=(5, 4))
print(X.shape)
# Training data
plt.scatter(X[:,1], y, color='blue', label=feat_target)
plt.scatter(X[24:,1], y_pred, color='red', label=feat_pred)
plt.title('Training Data')
plt.xlabel(sat_data)
plt.ylabel(feat_target)
plt.legend()
plt.grid(True)
plt.show()

data_ = data.iloc[24:]
data_[feat_pred] = y_pred

start_date = '2023-05-17'
end_date = '2023-05-20'

data.iloc[24:][feat_target][start_date:end_date].resample('H').mean().plot(figsize=(5, 4),label=feat_target)
data.iloc[24:][sat_data][start_date:end_date].resample('H').mean().plot(figsize=(5, 4),label=sat_data)
data_[feat_pred][start_date:end_date].resample('H').mean().plot(figsize=(5, 4),label=feat_pred)
plt.legend()
plt.title('Training Data')
plt.ylabel(y_label)
plt.xlabel('Date')
plt.grid(True)
plt.show()


In [ ]:
from keras.models import load_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler

model = load_model('/home/ahilan/gen-research/neoen-site/models/weather_prediction/Reverse_temp_lstm_atten_best_model_epoch_30_val_loss_0.08.h5')

X = df_met_nasa_temp[['Wind_nasa', 'Temp_nasa', 'POA_nasa', 'HUMDY_nasa']].values

y = data['Temp_mes'].values

# Normalize the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))


## Prepare the data for LSTM (convert to sequences)
def create_sequences(X, time_steps=1):
    Xs = []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
    return np.array(Xs)

seq_len = 24  # You can adjust this based on your data
X_seq = create_sequences(X_scaled, seq_len)

y_pred = scaler_y.inverse_transform(model.predict(X_seq))


df_met_nasa_temp_ = df_met_nasa_temp.iloc[24:]
df_met_nasa_temp_['Tmod_pred'] = y_pred

start_date = '2023-05-17'
end_date = '2023-05-20'

#start_date_sat = df_met_nasa_temp.index.min()
#end_date_sat = df_met_nasa_temp.index.max()

start_date_mes = data.index.min()
end_date_mes = data.index.max()

#df_met_nasa_temp['Wind_nasa'][start_date_sat:end_date_sat].resample('H').mean().plot(figsize=(5, 4),label='Sat Wind')
df_met_nasa_temp_['Temp_pred'][start_date_sat:end_date_sat].resample('H').mean().plot(figsize=(5, 4),label='Predicted Tmod')
data['Temp_mes'][start_date_mes:end_date_mes].resample('H').mean().plot(figsize=(5, 4),label='Measured Tmod')
plt.legend()
plt.title('Training Data')
plt.ylabel('Tmod')
plt.xlabel('Date')
plt.grid(True)
plt.show()


In [ ]:
df_met_nasa_temp_ = df_met_nasa_temp.iloc[24:]
df_met_nasa_temp_['Tmod_pred'] = y_pred_tmod
df_met_nasa_temp_['Temp_pred'] = y_pred_temp
df_met_nasa_temp_['Wind_pred'] = y_pred_wind
df_met_nasa_temp_['POA_pred'] = y_pred_POA
df_met_nasa = df_met_nasa_temp_
#Saving the nasa date on pkl file
df_met_nasa.to_pickle('/home/ahilan/gen-research/neoen-site/data/neoen_weather_nasa_sat_data_and_predicted_data.pkl')

In [ ]:
#Estimating the PR and WCPR on measured 1 year data
df_pow = pd.read_pickle('/mnt/data-r6/ahilan/dataset/neoen/process/pow_inv-32-A_5min.pkl')
df_POA = pd.read_pickle('/mnt/data-r6/ahilan/dataset/neoen/process/poa_5min.pkl')
df_Tmod = pd.read_pickle('/mnt/data-r6/ahilan/dataset/neoen/process/tmod_5min.pkl')
df_met = pd.read_pickle('/mnt/data-r6/ahilan/dataset/neoen/process/met_5min.pkl')
df_T = df_met[['Temp','Timestamp']]
df_T = df_T.rename(columns={'Temp': 'Temp_mes','Timestamp': 'time'})

#del df_neoen_measured_data
#del df_met_nasa_predicted
#del df_neoen_data_nasa_predicted_data

df_inner = pd.merge(df_pow, df_POA, on='time', how='inner')
df_inner2 = pd.merge(df_inner, df_Tmod, on='time', how='inner')
df = pd.merge(df_inner2, df_T, on='time', how='inner')


df.set_index('time', inplace=True)
df_neoen_measured_data = df.rename(columns={'power': 'Power_mes','value_x': 'POA_mes','value_y': 'Tmod_mes' })

df_neoen_mes = df_neoen_measured_data[['Power_mes','Temp_mes','POA_mes','Tmod_mes']].resample('H').mean()
#df_neoen_mes = df_neoen_measured_data[df_neoen_measured_data['POA_mes']>0]
#df_neoen_mes = df_neoen_mes[df_neoen_mes['Tmod_mes']>0]



df_pow.set_index('time', inplace=True)
df_neoen_measured_power = df_pow[['power']].resample('H').mean()

df_met_nasa_predicted = pd.read_pickle('/home/ahilan/gen-research/neoen-site/data/neoen_weather_nasa_sat_data_and_predicted_data.pkl')

df_met_nasa = df_met_nasa_predicted[['POA_nasa','Tmod_nasa']]
#df_met_nasa = df_met_nasa[df_met_nasa['POA_nasa']>0]
#df_met_nasa = df_met_nasa[df_met_nasa['Tmod_nasa']>0]

df_met_pred = df_met_nasa_predicted[['POA_pred','Tmod_pred']]
#df_met_pred = df_met_pred[df_met_pred['POA_pred']>0]
#df_met_pred = df_met_pred[df_met_pred['Tmod_pred']>0]



df_neoen_nasa = pd.merge(df_neoen_measured_power, df_met_nasa, left_index=True, right_index=True)
df_neoen_pred = pd.merge(df_neoen_measured_power, df_met_pred, left_index=True, right_index=True)

In [ ]:
POA_std = 1000
#Each inverter each with an average of 5,700 modules for a 2 MW nominal maximum power input
#with output AC power limited to 1.7 MW

Capacity = 2.0

df_neoen_mes['PR_mes_data'] = (df_neoen_mes['Power_mes'] * POA_std) / (Capacity * df_neoen_mes['POA_mes'])

T_std = 25 
#T_avg = df['Tmod'].mean()
alpha = 0.004

correction_cofficient = (1 - alpha * (T_std - df_neoen_mes['Tmod_mes']))
df_neoen_mes['WCPR_mes_data'] = df_neoen_mes['PR_mes_data']/correction_cofficient


df_neoen_nasa['PR_nasa_data'] = (df_neoen_nasa['power'] * POA_std) / (Capacity * df_neoen_nasa['POA_nasa'])

T_std = 25 
#T_avg = df['Tmod'].mean()
alpha = 0.004

correction_cofficient = (1 - alpha * (T_std - df_neoen_nasa['Tmod_nasa']))
df_neoen_nasa['WCPR_nasa_data'] = df_neoen_nasa['PR_nasa_data']/correction_cofficient


df_neoen_pred['PR_pred_data'] = (df_neoen_pred['power'] * POA_std) / (Capacity * df_neoen_pred['POA_pred'])

T_std = 25 
#T_avg = df['Tmod'].mean()
alpha = 0.004

correction_cofficient = (1 - alpha * (T_std - df_neoen_pred['Tmod_pred']))
df_neoen_pred['WCPR_pred_data'] = df_neoen_pred['PR_pred_data']/correction_cofficient


In [ ]:
WCPR_mes = df_neoen_mes[df_neoen_mes['WCPR_mes_data'] > 0.7]
WCPR_mes = WCPR_mes[WCPR_mes['WCPR_mes_data'] <= 1.2]

WCPR_nasa = df_neoen_nasa[df_neoen_nasa['WCPR_nasa_data'] > 0.7]
WCPR_nasa = WCPR_nasa[WCPR_nasa['WCPR_nasa_data'] <= 1.2]

WCPR_pred = df_neoen_pred[df_neoen_pred['WCPR_pred_data'] > 0.7]
WCPR_pred = WCPR_pred[WCPR_pred['WCPR_pred_data'] <= 1.2]

start_date = WCPR_nasa.index.min()
end_date = WCPR_nasa.index.max()
print(start_date)

#WCPR_measured_filtered = WCPR_measured_filtered[start_date:end_date]['WCPR_mes_data'].resample('D').mean()

plt.figure(figsize=(15, 6))
#plt.scatter(WCPR_measured_filtered['WCPR_mes_data'].index.date, WCPR_measured_filtered['WCPR_mes_data'])

WCPR_mes['WCPR_mes_data'][start_date:end_date].resample('D').mean().plot(figsize=(15, 6))
WCPR_mes['WCPR_mes_data'][start_date:end_date].resample('M').mean().plot(figsize=(15, 6))
WCPR_mes['WCPR_mes_data'][start_date:end_date].resample('Y').mean().plot(figsize=(15, 6))


# Format x-axis to show every hour
plt.gca().xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y:%m:%d'))
plt.title('Daily WCPR variation on Neoen weather station data')
plt.xlabel('Time (Days)')
plt.ylabel('WCPR on Neoen weather station data')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.grid(True)
plt.show()
print(np.mean(WCPR_mes['WCPR_mes_data'][start_date:end_date].resample('Y').mean()))

##############################################################################################################

#start_date = WCPR_nasa_filtered.index.min()
#end_date = WCPR_nasa_filtered.index.max()

#WCPR_measured_filtered = WCPR_measured_filtered[start_date:end_date]['WCPR_mes_data'].resample('D').mean()

plt.figure(figsize=(15, 6))
#plt.scatter(WCPR_nasa_filtered['WCPR_nasa_data'].index.date, WCPR_nasa_filtered['WCPR_nasa_data'])

WCPR_nasa['WCPR_nasa_data'][start_date:end_date].resample('D').mean().plot(figsize=(15, 6))
WCPR_nasa['WCPR_nasa_data'][start_date:end_date].resample('M').mean().plot(figsize=(15, 6))
WCPR_nasa['WCPR_nasa_data'][start_date:end_date].resample('Y').mean().plot(figsize=(15, 6))


# Format x-axis to show every hour
plt.gca().xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y:%m:%d'))
plt.title('Daily WCPR variation on nasa weather data')
plt.xlabel('Time (Days)')
plt.ylabel('WCPR on nasa weather data')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.grid(True)
plt.show()
print(np.mean(WCPR_nasa['WCPR_nasa_data'][start_date:end_date].resample('Y').mean()))

##############################################################################################################

#start_date = WCPR_pred_filtered.index.min()
#end_date = WCPR_pred_filtered.index.max()

#WCPR_measured_filtered = WCPR_measured_filtered[start_date:end_date]['WCPR_mes_data'].resample('D').mean()

plt.figure(figsize=(15, 6))
#plt.scatter(WCPR_nasa_filtered['WCPR_nasa_data'].index.date, WCPR_nasa_filtered['WCPR_nasa_data'])

WCPR_pred['WCPR_pred_data'][start_date:end_date].resample('D').mean().plot(figsize=(15, 6))
WCPR_pred['WCPR_pred_data'][start_date:end_date].resample('M').mean().plot(figsize=(15, 6))
WCPR_pred['WCPR_pred_data'][start_date:end_date].resample('Y').mean().plot(figsize=(15, 6))


# Format x-axis to show every hour
plt.gca().xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y:%m:%d'))
plt.title('Daily WCPR variation on weather predicted data')
plt.xlabel('Time (Days)')
plt.ylabel('WCPR on predicted weather data')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.grid(True)
plt.show()
print(np.mean(WCPR_pred['WCPR_pred_data'][start_date:end_date].resample('Y').mean()))


In [ ]:
'''
# CNN-LSTM attention modelling to predict actual temperature from satellite data temperature

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score


data = data[['Temp_mes','Wind_mes','POA_mes','Wind_nasa', 'Temp_nasa', 'POA_nasa', 'HUMDY_nasa']]

#data = data[data['Wind_mes']<12]
#data = data[data['Wind_nasa']<12]

# Filtering constant values
# Define the window size
window_size = 3

# Check if the values in the rolling window are the same
data['constant'] = data['Wind_mes'].rolling(window=window_size).apply(lambda x: np.all(x == x[0]), raw=True)

# Fill NaN values with 0 (occurs for the first 'window_size - 1' elements)
data['constant'].fillna(0, inplace=True)
data['constant'] = data['constant'].astype(bool)

# Filter out constant value rows
data_filtered = data[~data['constant']].drop(columns=['constant'])


data = data_filtered

start_date = data.index.min()
end_date = data.index.max()

feat_target = 'Wind_mes'
sat_data = 'Wind_nasa'
feat_pred = 'Wind_pred'
y_label = 'Wind'

# Split the data into features (X) and target (y)
X = data[['Wind_nasa', 'Temp_nasa', 'POA_nasa', 'HUMDY_nasa']].values
y = data[feat_target].values


# Normalize the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Prepare the data for LSTM (convert to sequences)
def create_sequences(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

seq_len = 24  # You can adjust this based on your data
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_len)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

hybrid_model = cnn_lstm_atten_train(X_train, X_test, y_train, y_test, feat_target, sat_data, feat_pred, y_label,epochs=50)

# Evaluate the model
mse, mae = hybrid_model.evaluate(X_test, y_test)
print(f"Test MSE: {mse}")
print(f"Test MAE: {mae}")

# Make predictions
y_train_pred = hybrid_model.predict(X_train)
y_test_pred = hybrid_model.predict(X_test)


# Calculate metrics
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Train MSE: {train_mse}')
print(f'Test MSE: {test_mse}')
print(f'Train R2: {train_r2}')
print(f'Test R2: {test_r2}')

y_pred = scaler_y.inverse_transform(hybrid_model.predict(X_seq))


plt.plot(y[24:84], color='blue', label=feat_target)
plt.plot(X[24:84,1], color='red', label=sat_data)
plt.plot(y_pred[0:60], color='green', label=feat_pred)
plt.title('POA Data')
plt.xlabel('Time')
plt.ylabel(y_label)
plt.legend()
plt.show()

# Plot the results
plt.figure(figsize=(5, 4))
print(X.shape)
# Training data
plt.scatter(X[:,1], y, color='blue', label=feat_target)
plt.scatter(X[24:,1], y_pred, color='red', label=feat_pred)
plt.title('Training Data')
plt.xlabel(sat_data)
plt.ylabel(feat_target)
plt.legend()
plt.grid(True)
plt.show()

data_ = data.iloc[24:]
data_[feat_pred] = y_pred

start_date = '2023-05-17'
end_date = '2023-05-20'

data.iloc[24:][feat_target][start_date:end_date].resample('H').mean().plot(figsize=(5, 4),label=feat_target)
data.iloc[24:][sat_data ][start_date:end_date].resample('H').mean().plot(figsize=(5, 4),label=sat_data)
data_[feat_pred][start_date:end_date].resample('H').mean().plot(figsize=(5, 4),label=feat_pred)
plt.legend()
plt.title('Training Data')
plt.ylabel(y_label)
plt.xlabel('Date')
plt.grid(True)
plt.show()
'''